In [1]:
import os

from typing import Annotated
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore

load_dotenv()

True

In [2]:
# Constants
user_Id = "1000"
thread_Id = "203"
memory_namespace = (user_Id, "memories")

### Adding semantic search to memory

In [3]:

# Create store with semantic search enabled
embeddings = init_embeddings(
    "azure_openai:text-embedding-ada-002", 
    openai_api_key=os.getenv("AZURE_TEXT_EMBEDDING_ADA_002_KEY")
)

store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
    }
)

# To update the store with new memories
store.put(memory_namespace, "1", {"text": "User name is Chandan"})
store.put(memory_namespace, "2", {"text": "User loves pizza"})

# To search for memories. This will use semantic search
# and return the most relevant memory based on the query.
memories = store.search(
    memory_namespace, query="I have leak in my pipe", limit=1
)

for memory in memories:
    print(f"Memory: {memory.value['text']}, Score: {memory.score}")
    
# Note: Require Numpy for vector 
# https://langchain-ai.github.io/langgraph/how-tos/memory/semantic-search/

Memory: User loves pizza, Score: 0.7170508266728534


### Consuming the Memory

In [ ]:

llm = init_chat_model(
    "azure_openai:gpt-4o-mini"
)

class State(TypedDict):
    messages: Annotated[list, add_messages]
    
def chatbot(state: State, config: RunnableConfig, *, store: BaseStore):    
    # Search based on the most recent message
    memories = store.search(
        memory_namespace,
        query=state["messages"][-1].content,
        limit=3
    )
    
    # Join the memory texts into a single string
    info = "\n".join([d.value["text"] for d in memories])
    
    # Build the system message with user info
    system_msg = f"You are a helpful assistant talking to the user. User info: {info}"
    
    # Invoke the LLM with the system message and user messages
    response = llm.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    
    return {"messages": response}

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("llmchatbot", chatbot)
graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END) 

checkpointer = InMemorySaver()

graph = graph_builder.compile(checkpointer=checkpointer, store=store)

while True:
    user_input = input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        break
    
    # Invoke the graph with the user message
    response = graph.invoke(
        {
            "messages": [{"role": "user", "content": user_input}],
        },
        {
            "configurable": {
                "thread_id": thread_Id,
                "user_id": user_Id
            }
        }
    )
    
    for msg in response["messages"]:
        print(f"Bot: {msg.content}")